# 🧪 Run Batch Experiment

This notebook runs multiple adversarial games with different configurations and analyzes the aggregated results.

## Experiment Matrix
Configure combinations of:
- **Models**: Different LLMs for Red/Blue teams
- **Difficulties**: easy, medium, hard
- **Scenarios**: Different infrastructure domains

## 1. Setup

In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from dotenv import load_dotenv
load_dotenv(project_root / '.env')

import asyncio
import json
import os
from datetime import datetime
import pandas as pd

from src.game.engine import GameEngine, GameConfig
from src.game.scenarios import ScenarioGenerator, Scenario
from src.agents.red_team_agent import Difficulty

print(f"✓ Project root: {project_root}")
print("✓ All imports successful!")

## 2. Configure Experiment

Define the experiment matrix - all combinations will be tested:

In [ ]:
# ============================================
# 🧪 EXPERIMENT CONFIGURATION
# ============================================

EXPERIMENT_NAME = "notebook_experiment"
AWS_REGION = os.getenv("AWS_REGION", "us-east-1")

# Models to test
MODELS = [
    {"id": "us.anthropic.claude-3-5-haiku-20241022-v1:0", "name": "Haiku"},
    # {"id": "us.anthropic.claude-3-5-sonnet-20241022-v2:0", "name": "Sonnet"},  # Uncomment for more tests
]

# Difficulty levels
DIFFICULTIES = ["medium", "hard"]

# Scenarios to test
SCENARIOS = [
    {"domain": "storage", "description": "Create S3 bucket for PHI data with security controls"},
    {"domain": "compute", "description": "Create Lambda function processing payment data"},
    {"domain": "iam", "description": "Create IAM roles for CI/CD pipeline"},
]

# Calculate total games
total_games = len(MODELS) * len(MODELS) * len(DIFFICULTIES) * len(SCENARIOS)
print(f"📊 Experiment: {EXPERIMENT_NAME}")
print(f"   Models: {len(MODELS)}")
print(f"   Difficulties: {len(DIFFICULTIES)}")  
print(f"   Scenarios: {len(SCENARIOS)}")
print(f"   Total games: {total_games}")

## 3. Generate Game Configurations

In [ ]:
# Generate all game configurations
game_configs = []

for red_model in MODELS:
    for blue_model in MODELS:
        for difficulty in DIFFICULTIES:
            for scenario in SCENARIOS:
                config = GameConfig(
                    red_model_id=red_model["id"],
                    blue_model_id=blue_model["id"],
                    difficulty=Difficulty[difficulty.upper()],
                    scenario=Scenario(
                        domain=scenario["domain"],
                        description=scenario["description"],
                        cloud_provider="aws",
                        language="terraform"
                    ),
                    region=AWS_REGION
                )
                game_configs.append({
                    "config": config,
                    "red_model_name": red_model["name"],
                    "blue_model_name": blue_model["name"],
                    "difficulty": difficulty,
                    "domain": scenario["domain"]
                })

print(f"✓ Generated {len(game_configs)} game configurations")

## 4. Run Experiment

⚠️ **This will make LLM API calls and may take several minutes**

In [ ]:
# Run all games
results = []
engine = GameEngine()

for i, game in enumerate(game_configs, 1):
    print(f"\n🎮 Running game {i}/{len(game_configs)}")
    print(f"   Red: {game['red_model_name']} | Blue: {game['blue_model_name']}")
    print(f"   Difficulty: {game['difficulty']} | Domain: {game['domain']}")
    
    try:
        result = await engine.play(game["config"])
        
        results.append({
            "game_id": i,
            "red_model": game["red_model_name"],
            "blue_model": game["blue_model_name"],
            "difficulty": game["difficulty"],
            "domain": game["domain"],
            "precision": result.scoring.metrics.precision,
            "recall": result.scoring.metrics.recall,
            "f1_score": result.scoring.metrics.f1_score,
            "evasion_rate": result.scoring.metrics.evasion_rate,
            "red_vulns": result.scoring.counts.total_red_vulns,
            "blue_findings": result.scoring.counts.total_blue_findings,
            "true_positives": result.scoring.counts.true_positives,
            "status": "success"
        })
        print(f"   ✓ F1: {result.scoring.metrics.f1_score:.1%}")
        
    except Exception as e:
        results.append({
            "game_id": i,
            "red_model": game["red_model_name"],
            "blue_model": game["blue_model_name"],
            "difficulty": game["difficulty"],
            "domain": game["domain"],
            "status": "failed",
            "error": str(e)
        })
        print(f"   ❌ Failed: {e}")

print(f"\n✓ Completed {len([r for r in results if r['status'] == 'success'])}/{len(game_configs)} games")

## 5. Analyze Results

In [ ]:
# Create DataFrame for analysis
df = pd.DataFrame([r for r in results if r["status"] == "success"])

if len(df) > 0:
    print("📊 EXPERIMENT RESULTS")
    print("=" * 60)
    
    # Overall metrics
    print(f"\n📈 Overall Metrics (n={len(df)}):")
    print(f"   Avg Precision:    {df['precision'].mean():.1%}")
    print(f"   Avg Recall:       {df['recall'].mean():.1%}")
    print(f"   Avg F1 Score:     {df['f1_score'].mean():.1%}")
    print(f"   Avg Evasion Rate: {df['evasion_rate'].mean():.1%}")
    
    # By difficulty
    print(f"\n📊 By Difficulty:")
    difficulty_stats = df.groupby('difficulty')[['f1_score', 'evasion_rate']].mean()
    print(difficulty_stats.to_string())
    
    # By domain
    print(f"\n📊 By Domain:")
    domain_stats = df.groupby('domain')[['f1_score', 'evasion_rate']].mean()
    print(domain_stats.to_string())
else:
    print("❌ No successful games to analyze")

# Display full results table
if len(df) > 0:
    display_cols = ['game_id', 'red_model', 'blue_model', 'difficulty', 'domain', 
                    'precision', 'recall', 'f1_score', 'evasion_rate']
    print("\n📋 Full Results Table:")
    print(df[display_cols].to_string(index=False))

In [ ]:
## 6. Save Experiment Results

# Save results
output_dir = project_root / "experiments" / f"{EXPERIMENT_NAME}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
output_dir.mkdir(parents=True, exist_ok=True)

# Save as JSON
with open(output_dir / "results.json", "w") as f:
    json.dump(results, f, indent=2, default=str)

# Save as CSV
if len(df) > 0:
    df.to_csv(output_dir / "results.csv", index=False)

print(f"✓ Results saved to: {output_dir}")